In [2]:
%%capture
import os
from random import seed
from model import single_image
from unrestricted_advex.eval_kit import logits_to_preds, _validate_logits
import mitsuba as mi
mi.set_variant('llvm_ad_rgb')
import drjit as dr
import torch.nn as nn
import torch
import torchvision.models as models
import numpy as np
import graphviz
import time

2022-09-13 18:00:49.830446: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Render Scene

In [3]:
def mitsuba_bike_scene() -> mi.llvm_ad_rgb.Scene:
    bike_scene = mi.load_file('../data_generation/scenes/bike/bike.xml', integrator='prb')
    params = mi.traverse(bike_scene)   
    params['mat-BodyPaint1_SG.001.reflectance.value'] = [0.15, 0.15, 0.6] # frame, fork
    params['mat-GlossBlack_SG.001.reflectance.value'] = [0.0, 0.0, 0.0] # spoke, rims
    params['mat-Silver1_SG.001.reflectance.value'] = [0.6, 0.6, 0.6] # chain, gears, rims, cranks
    params['mat-CableGray1_SG.001.reflectance.value'] = [0.4, 0.4, 0.4] # cables
    params['mat-Wire1_SG.001.reflectance.value'] = [0.1, 0.1, 0.1] # misc. wires
    params['mat-Silver2_SG.001.reflectance.value'] = [0.8, 0.8, 0.8] # pedals, handbrake levers
    params['mat-TireGom1_SG.001.reflectance.value'] = [0.0, 0.0, 0.0] #tires
    params['mat-GlossWhite_SG.001.reflectance.value'] = [0.0, 0.0, 0.0] # handlebars, seat
    params['mat-MatWhite1_SG.001.reflectance.value'] = [0.01, 0.01, 0.01] #handlebar covers
    params['mat-MatBlack_SG.001.reflectance.value'] = [0.0, 0.0, 0.0] # hand-brakes
    params['mat-Material.002.reflectance.value'] = [0.6, 0.6, 0.6]


    params.update()             
    return bike_scene

# orig_bike_scene = mitsuba_bike_scene()
# orig_bike_scene_params = mi.traverse(orig_bike_scene)    
# orig_bike_image = mi.render(scene=orig_bike_scene, params=orig_bike_scene_params, spp=512)
# mi.util.convert_to_bitmap(orig_bike_image)

In [4]:
def mitsuba_bird_scene_multi_cam() -> mi.llvm_ad_rgb.Scene:
    gray_bird_scene_path = '../data_generation/scenes/multi_cam_bird/gray_bird_multi_light.xml'
    gray_bird_scene = mi.load_file(gray_bird_scene_path)
    gray_bird_params = mi.traverse(gray_bird_scene)
    c = 0.85
    wall_color = mi.Color3f(c, c, c)
    b = 0.175
    bird_color = mi.Color3f(b, b, b)
    gray_bird_params['mat-Material.001.reflectance.value'] = bird_color # bike mesh
    gray_bird_params['mat-Material.002.reflectance.value'] = wall_color # plane - floor
    gray_bird_params['mat-Material.003.reflectance.value'] = wall_color # plane - front
    gray_bird_params['mat-Material.004.reflectance.value'] = wall_color # plane - back
    gray_bird_params['mat-Material.006.reflectance.value'] = wall_color # plane - left
    gray_bird_params['mat-Material.007.reflectance.value'] = wall_color # plane - right
    gray_bird_params['Rectangle.emitter.radiance.value'] = [.9, .9, .9] # light rgb
    gray_bird_params.update()
    return gray_bird_scene

# multi_cam_bird_scene = mitsuba_bird_scene_multi_cam() 
# multi_cam_bird_params = mi.traverse(multi_cam_bird_scene)   
# gray_bird_image = mi.render(scene=multi_cam_bird_scene, params=multi_cam_bird_params, sensor = 5, spp=512)
# mi.util.convert_to_bitmap(gray_bird_image)

In [5]:
def mitsuba_bird_2_scene_multi_cam() -> mi.llvm_ad_rgb.Scene:
    gray_bird_scene_path = '../data_generation/scenes/multi_cam_bird_2/gray_bird_2_multi_light.xml'
    gray_bird_scene = mi.load_file(gray_bird_scene_path)
    gray_bird_params = mi.traverse(gray_bird_scene)
    c = 0.85
    wall_color = mi.Color3f(c, c, c)
    b = 0.175
    bird_color = mi.Color3f(b, b, b)
    gray_bird_params['mat-12214_bird.reflectance.value'] = bird_color # bike mesh
    gray_bird_params['mat-Material.002.reflectance.value'] = wall_color # plane - floor
    gray_bird_params['mat-Material.003.reflectance.value'] = wall_color # plane - front
    gray_bird_params['mat-Material.004.reflectance.value'] = wall_color # plane - back
    gray_bird_params['mat-Material.006.reflectance.value'] = wall_color # plane - left
    gray_bird_params['mat-Material.007.reflectance.value'] = wall_color # plane - right
    gray_bird_params['Rectangle.emitter.radiance.value'] = [.9, .9, .9] # light rgb
    gray_bird_params.update()
    return gray_bird_scene

# multi_cam_bird_2_scene = mitsuba_bird_2_scene_multi_cam() 
# multi_cam_bird_2_params = mi.traverse(multi_cam_bird_2_scene)   
# gray_bird_2_image = mi.render(scene=multi_cam_bird_2_scene, params=multi_cam_bird_2_params, sensor = 6, spp=512)
# mi.util.convert_to_bitmap(gray_bird_2_image)

In [6]:
def mitsuba_bike_scene_multi_cam() -> mi.llvm_ad_rgb.Scene:
    gray_bike_scene_path = '../data_generation/scenes/multi_cam_bike/gray_bike_multi_light.xml'
    gray_bike_scene = mi.load_file(gray_bike_scene_path)
    gray_bike_params = mi.traverse(gray_bike_scene)
    c = 0.85
    wall_color = mi.Color3f(c, c, c)
    b = 0.2
    bike_color = mi.Color3f(b, b, b)
    gray_bike_params['mat-default.reflectance.value'] = bike_color # bike mesh
    gray_bike_params['mat-Material.002.reflectance.value'] = wall_color # plane - floor
    gray_bike_params['mat-Material.003.reflectance.value'] = wall_color # plane - front
    gray_bike_params['mat-Material.004.reflectance.value'] = wall_color # plane - back
    gray_bike_params['mat-Material.006.reflectance.value'] = wall_color # plane - left
    gray_bike_params['mat-Material.007.reflectance.value'] = wall_color # plane - right
    gray_bike_params['Rectangle.emitter.radiance.value'] = [.9, .9, .9] # light rgb
    gray_bike_params.update()
    return gray_bike_scene

# multi_cam_bike_scene = mitsuba_bike_scene_multi_cam() 
# multi_cam_bike_params = mi.traverse(multi_cam_bike_scene)   
# gray_bike_image = mi.render(scene=multi_cam_bike_scene, params=multi_cam_bike_params, sensor = 7, spp=512)
# mi.util.convert_to_bitmap(gray_bike_image)

In [7]:
# load pre-trained bird-or-bicycle model
checkpoint = torch.load('/nvmescratch/mhull32/unrestricted-adversarial-examples/model_zoo/undefended_pytorch_resnet.pth.tar')
model = getattr(models, checkpoint['arch'])(num_classes=2)
model = nn.Sequential(nn.BatchNorm2d(num_features=3, affine=False), model)
model = torch.nn.DataParallel(model).cuda()
model.load_state_dict(checkpoint['state_dict'])
target = torch.tensor([1]).cuda() # 0 for bike, 1 for bird
criterion = nn.CrossEntropyLoss()

model.eval()
torch.set_grad_enabled(True)   

Convert to Image

Preprocess Image for Classification

Get Model Outputs for the Rendered Bike Scene

Calculate the Loss of Model Prediction with Respect to Label

Save the Gradient

Take the gradient of the loss with respect to the input image and assign it to the _rendered image_ from Mitsuba3

Backpropogate the gradient of the loss wrt the input image back through the renderer

In [29]:
from importlib.metadata import requires


def attack_mesh(scene:mi.llvm_ad_rgb.Scene=None, iterations:int=None, epsilon:float=None, alpha:float=None, target_mesh_keys:list=None, **kwargs): 
    print(f"Running attack with eps: {epsilon}, alpha: {alpha}, iters: {iterations}")  
    orig_params = mi.traverse(scene)   
    orig_mesh_vertex_positions = [dr.llvm.ad.Float(orig_params[k]) for k in target_mesh_keys] 
    sensor = 0
    if 'sensor' in kwargs.keys():
        sensor = kwargs['sensor']
    else:
        pass
    print(f'rendering using sensor {sensor}.')
    
    scene_params = mi.traverse(scene)   
    for target_mesh in target_mesh_keys:
        dr.enable_grad(scene_params[target_mesh])
    scene_params.update()
    
    for it in range(iterations):
        print(f"iter: {it}")
        image_ref = mi.render(scene, scene_params, sensor=sensor, spp=256, seed=999) 
        im_path = f'adv_tmp_im_{it}.jpg'   
        mi.util.write_bitmap(im_path, image_ref)
        im = np.array([image_ref])
        x_np = im.transpose((0, 3, 1, 2))  # from NHWC to NCHW
        x = torch.tensor(x_np, requires_grad=True)
        logits = model(x)
        loss = criterion(logits, target).requires_grad_()
        loss.backward()
        _x_grad = x.grad.detach().clone()
        rendered_im_grad = mi.TensorXf(_x_grad.permute(0, 3, 2, 1)[0])
        model.zero_grad()
        correct = np.equal(logits_to_preds(logits.detach().cpu().numpy()), target.cpu().numpy()).astype(np.float32)
        print(f"corect: {correct}, loss: {loss}")
        if dr.grad_enabled(image_ref) == False:
            dr.enable_grad(image_ref)
        dr.set_grad(image_ref, rendered_im_grad)
        dr.backward(image_ref)
        
        # attack meshes we are interested in
        print(f"perturbing vertices...")
        for i, k in enumerate(target_mesh_keys):
            # the gradient of the output image with respect to the mesh
            mesh_grad = dr.grad(scene_params[k])
            # update if the gradient is not zero, or skip
            if len(np.where(np.array(mesh_grad) > 0)[0]) == 0:
                continue    
            else:
                # l-inf attack, vertex positions + sign of the grad * the learning rate (α)
                scene_params[k] = scene_params[k] + dr.sign(mesh_grad) * alpha
                delta = scene_params[k] - orig_mesh_vertex_positions[i]
                # restrict perturbation to remain within ε budget
                delta = dr.clamp(delta, -epsilon, epsilon)
                # note: updating the value automatically zeros the grad
                scene_params[k] = orig_mesh_vertex_positions[i] + delta
            scene_params.update() 


# keys = ['bike_mesh.vertex_positions']    
# keys = ['bird_mesh.vertex_positions']    
# keys = ['yellow_bird_mesh.vertex_positions']        
# number of iterations * alpha >= epsilon
# alpha = epsilon / iters
# .006274 = .06274 / 10

iters = 4
epsilon = 32/255
alpha = epsilon / (iters)
target_mesh_keys = ['yellow_bird_mesh.vertex_positions']
# bike_scene = mitsuba_bike_scene_multi_cam()
bird_scene = mitsuba_bird_2_scene_multi_cam()
attack_mesh(scene=bird_scene, iterations=iters, epsilon=epsilon, alpha=alpha, target_mesh_keys=target_mesh_keys, sensor=6)

Running attack with eps: 0.12549019607843137, alpha: 0.03137254901960784, iters: 4
rendering using sensor 6.
iter: 0
corect: [1.], loss: 0.07054895907640457
perturbing vertices...
iter: 1
corect: [1.], loss: 0.033616840839385986
perturbing vertices...
iter: 2
corect: [1.], loss: 0.03361476585268974
perturbing vertices...
iter: 3
corect: [1.], loss: 0.04008102044463158
perturbing vertices...


### Run Attack

In [10]:
# extract a 'perturbed' mesh
# extract and write out a mesh from a scene
# use the original .ply to find # vertices, faces

p = mi.traverse(bird_scene)
mesh = mi.Mesh(
    "perturbed_bird",
    vertex_count=1006794,
    face_count=1994752 - 1,
    has_vertex_normals=False,
    has_vertex_texcoords=False,
)
mesh_params = mi.traverse(mesh)
#print(mesh_params)
mesh_params["vertex_positions"] = dr.ravel(p['yellow_bird_mesh.vertex_positions'])
mesh_params["faces"] = dr.ravel( p['yellow_bird_mesh.faces'])
print(mesh_params.update())
mesh.write_ply("perturbed_bird.ply")


[(Mesh[
  name = "perturbed_bird",
  bbox = BoundingBox3f[
    min = [-2.22926, -0.119617, -0.958572],
    max = [1.8862, 2.62876, 0.957979]
  ],
  vertex_count = 1006794,
  vertices = [11.5 MiB of vertex data],
  face_count = 1994751,
  faces = [22.8 MiB of face data],
  face_normals = 0
], {'faces', 'vertex_positions'})]


# TODO
- Show effect of gradients - which meshes receive gradients vs. those that do not, when we backprop classifer loss --> mitsuba  ?
    - Visualize by re-render scene heat map to show which meshes have gradient by showing custom material on meshes w/ non-zero grad?
    - Visualize by forward propogation of the gradient of the mesh to the output image and then plot gradient image
- Does the gradient change when we change camera angle? Insert more sensors into the scene, and then re-run pipeline to watch gradient change
- Can achieve expectation over transformation by producing perturbed mesh by rotating camera at different angles and then perturbing the scene based on the gradient produced at various angles?


In [ ]:
"""
Import scene 
Attack Loop
    Render Scene 
    Convert to image
    Preproccess Image for classification
    Get Model Outputs
    Calculate Loss of Model Prediction with Respect to Label
    Backpropogate the loss through the renderer
    Get the Gradient of the loss with respect to the input parameters
    Take the sign of the gradient and apply to the input parameter
    Update the scene Parameters
    Render Scene
"""